### 🖋 **Notebook Contents**

0. Initial Setup
1. Modelling
2. Conclusion
3. Recommendation

****

## `Initial Setup`

In [1]:
# Data Manipulation
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import missingno as msno

import warnings
warnings.filterwarnings("ignore")

# Model Algorithm (modeling)
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from xgboost.sklearn import XGBRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn import linear_model
import statsmodels.api as sm

# Data Preparation
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV, GridSearchCV, StratifiedKFold, KFold
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder
import category_encoders as ce
from sklearn.compose import TransformedTargetRegressor

# Evaluation metrics
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error, mean_squared_log_error, r2_score

# pycaret
import statistics

### Main Data

In [2]:
data = pd.read_csv('..\data\processed\salaries_clean.csv')
data

,work_year,experience_level,employment_type,salary_in_usd,employee_residence,remote_ratio,company_location,company_size,job_position,job_scope
0,2023,SE,FT,132000,US,100,US,M,STAFF,DATA ENGINEER
1,2023,MI,FT,81206,GB,0,GB,M,STAFF,ML/AI ENGINEER
2,2023,EX,FT,330000,US,0,US,M,HEAD,ML/AI ENGINEER
3,2023,EX,FT,188000,US,0,US,M,HEAD,ML/AI ENGINEER
4,2023,MI,FT,140000,US,0,US,M,STAFF,BUSINESS INTELLIGENCE
...,...,...,...,...,...,...,...,...,...,...
4245,2020,SE,FT,412000,US,100,US,L,STAFF,DATA SCIENTIST
4246,2021,MI,FT,151000,US,100,US,L,STAFF,DATA SCIENTIST
4247,2020,EN,FT,105000,US,100,US,S,STAFF,DATA SCIENTIST
4248,2020,EN,CT,100000,US,100,US,L,STAFF,DATA ANALYST


### Additional Data

In [3]:
geo = pd.read_csv(r'..\data\raw\ISO-3166-Countries-with-Regional-Codes.csv')
geo

,name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,NaN,142.0,34.0,NaN
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,NaN,150.0,154.0,NaN
2,Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,NaN,150.0,39.0,NaN
3,Algeria,DZ,DZA,12,ISO 3166-2:DZ,Africa,Northern Africa,NaN,2.0,15.0,NaN
4,American Samoa,AS,ASM,16,ISO 3166-2:AS,Oceania,Polynesia,NaN,9.0,61.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
244,Wallis and Futuna,WF,WLF,876,ISO 3166-2:WF,Oceania,Polynesia,NaN,9.0,61.0,NaN
245,Western Sahara,EH,ESH,732,ISO 3166-2:EH,Africa,Northern Africa,NaN,2.0,15.0,NaN
246,Yemen,YE,YEM,887,ISO 3166-2:YE,Asia,Western Asia,NaN,142.0,145.0,NaN
247,Zambia,ZM,ZMB,894,ISO 3166-2:ZM,Africa,Sub-Saharan Africa,Eastern Africa,2.0,202.0,14.0


### Merge Data

In [4]:
data = data.merge(geo[['alpha-2','region']], left_on='company_location', right_on='alpha-2', how='left').drop(columns='alpha-2').rename(columns={'region':'company_region'})
data

,work_year,experience_level,employment_type,salary_in_usd,employee_residence,remote_ratio,company_location,company_size,job_position,job_scope,company_region
0,2023,SE,FT,132000,US,100,US,M,STAFF,DATA ENGINEER,Americas
1,2023,MI,FT,81206,GB,0,GB,M,STAFF,ML/AI ENGINEER,Europe
2,2023,EX,FT,330000,US,0,US,M,HEAD,ML/AI ENGINEER,Americas
3,2023,EX,FT,188000,US,0,US,M,HEAD,ML/AI ENGINEER,Americas
4,2023,MI,FT,140000,US,0,US,M,STAFF,BUSINESS INTELLIGENCE,Americas
...,...,...,...,...,...,...,...,...,...,...,...
4245,2020,SE,FT,412000,US,100,US,L,STAFF,DATA SCIENTIST,Americas
4246,2021,MI,FT,151000,US,100,US,L,STAFF,DATA SCIENTIST,Americas
4247,2020,EN,FT,105000,US,100,US,S,STAFF,DATA SCIENTIST,Americas
4248,2020,EN,CT,100000,US,100,US,L,STAFF,DATA ANALYST,Americas


In [5]:
data['aboard'] = data['employee_residence'] != data['company_location']
data

,work_year,experience_level,employment_type,salary_in_usd,employee_residence,remote_ratio,company_location,company_size,job_position,job_scope,company_region,aboard
0,2023,SE,FT,132000,US,100,US,M,STAFF,DATA ENGINEER,Americas,False
1,2023,MI,FT,81206,GB,0,GB,M,STAFF,ML/AI ENGINEER,Europe,False
2,2023,EX,FT,330000,US,0,US,M,HEAD,ML/AI ENGINEER,Americas,False
3,2023,EX,FT,188000,US,0,US,M,HEAD,ML/AI ENGINEER,Americas,False
4,2023,MI,FT,140000,US,0,US,M,STAFF,BUSINESS INTELLIGENCE,Americas,False
...,...,...,...,...,...,...,...,...,...,...,...,...
4245,2020,SE,FT,412000,US,100,US,L,STAFF,DATA SCIENTIST,Americas,False
4246,2021,MI,FT,151000,US,100,US,L,STAFF,DATA SCIENTIST,Americas,False
4247,2020,EN,FT,105000,US,100,US,S,STAFF,DATA SCIENTIST,Americas,False
4248,2020,EN,CT,100000,US,100,US,L,STAFF,DATA ANALYST,Americas,False


In [6]:
# save data

data.to_csv("../data/processed/modeling_used.csv", index=False)

## `Modelling`

### Splitting

In [7]:
# divide feature and target

target = 'salary_in_usd'
feature = data.drop(columns=[target]).columns

display(target, feature)

'salary_in_usd'

Index(['work_year', 'experience_level', 'employment_type',
       'employee_residence', 'remote_ratio', 'company_location',
       'company_size', 'job_position', 'job_scope', 'company_region',
       'aboard'],
      dtype='object')

In [8]:
# data spliting

train, test = train_test_split(
    data,
    test_size = 0.20,
    random_state = 7
)

print(train.shape, test.shape)

(3400, 12) (850, 12)


### Encoding

In [9]:
cat = data.select_dtypes(object).columns

# check initial info of data
pd.DataFrame({
    'column': data[cat].columns.values,
    'type': data[cat].dtypes.values,
    'n_unique': data[cat].nunique().values,
    'min': data[cat].min().values,
    'max': data[cat].max().values,
    'sample_unique': [data[col].sort_values().unique() for col in data[cat].columns]
})

,column,type,n_unique,min,max,sample_unique
0,experience_level,object,4,EN,SE,"[EN, EX, MI, SE]"
1,employment_type,object,4,CT,PT,"[CT, FL, FT, PT]"
2,employee_residence,object,84,AD,ZA,"[AD, AE, AM, AR, AS, AT, AU, BA, BE, BG, BO, B..."
3,company_location,object,72,AD,ZA,"[AD, AE, AM, AR, AS, AT, AU, BA, BE, BR, BS, C..."
4,company_size,object,3,L,S,"[L, M, S]"
5,job_position,object,5,DIRECTOR,STAFF,"[DIRECTOR, HEAD, LEAD, MANAGER, STAFF]"
6,job_scope,object,9,ANALYTICS ENGINEER,RESEARCH/APPLIED SCIENTIST,"[ANALYTICS ENGINEER, BUSINESS INTELLIGENCE, DA..."
7,company_region,object,5,Africa,Oceania,"[Africa, Americas, Asia, Europe, Oceania]"


In [10]:
# divide encode handling
ohe = ['employment_type','job_scope']
oren = ['experience_level', 'company_size', 'job_position']
tgen = ['employee_residence', 'company_location','company_region']

display(ohe,oren,tgen)

['employment_type', 'job_scope']

['experience_level', 'company_size', 'job_position']

['employee_residence', 'company_location', 'company_region']

In [11]:
orenMap = [{'col': 'experience_level', 'mapping': {'EN': 0, 'MI': 1, 'SE': 2, 'EX': 3}}, 
     {'col': 'company_size', 'mapping': {'S': 0, 'M': 1, 'L': 2}},
     {'col': 'job_position', 'mapping': {'STAFF': 0, 'LEAD': 1, 'MANAGER': 2, 'HEAD': 3, 'DIRECTOR': 4}}]

pd.DataFrame(orenMap)

,col,mapping
0,experience_level,"{'EN': 0, 'MI': 1, 'SE': 2, 'EX': 3}"
1,company_size,"{'S': 0, 'M': 1, 'L': 2}"
2,job_position,"{'STAFF': 0, 'LEAD': 1, 'MANAGER': 2, 'HEAD': ..."


In [12]:
# Create encoder
ct = ColumnTransformer([
    ('One Hot Encoder', OneHotEncoder(drop='first'), ohe),
    ('Ordinal Encoder', ce.OrdinalEncoder(cols=oren, mapping=orenMap), oren),
    ('Target Encoder', ce.TargetEncoder(), tgen)
], remainder='passthrough')

### BenchMark Model

In [13]:
# Create Model

# Model without transformed target
# Stand Alone Model
lr = LinearRegression()
knn = KNeighborsRegressor()
dt = DecisionTreeRegressor(random_state=7)
# Ensemble Model
rf = RandomForestRegressor(random_state=7)
xgb = XGBRegressor(random_state=7)
ada = AdaBoostRegressor(random_state=7)


# Model with transformed target
# Stand Alone Model
log_lr = TransformedTargetRegressor(lr, func=np.log, inverse_func=np.exp)
log_knn = TransformedTargetRegressor(knn, func=np.log, inverse_func=np.exp)
log_dt = TransformedTargetRegressor(dt, func=np.log, inverse_func=np.exp)
# Ensemble Model
log_rf = TransformedTargetRegressor(rf, func=np.log, inverse_func=np.exp)
log_xgb = TransformedTargetRegressor(xgb, func=np.log, inverse_func=np.exp)
log_ada = TransformedTargetRegressor(ada, func=np.log, inverse_func=np.exp)

# Collect Models
models = {
    'LinearRegression': lr,
    'KNeighborsRegressor': knn,
    'DecisionTreeRegressor': dt,
    'RandomForestRegressor': rf,
    'XGBRegressor': xgb,
    'AdaBoostRegressor': ada,
    'LinearRegression-logTarget': log_lr,
    'KNeighborsRegressor-logTarget': log_knn,
    'DecisionTreeRegressor-logTarget': log_dt,
    'RandomForestRegressor-logTarget': log_rf,
    'XGBRegressor-logTarget': log_xgb,
    'AdaBoostRegressor-logTarget': log_ada,
    }

result = []

for name, est in models.items():
    
    crossval = KFold(n_splits=5, shuffle=True, random_state=7)

    estimator = Pipeline([
        ('preproces', ct),
        ('model', est)
    ])

    # MAE
    mae = cross_val_score(
        estimator, 
        train[feature], 
        train[target], 
        cv=crossval, 
        scoring='neg_mean_absolute_error'
        )

    # MAPE
    mape = cross_val_score(
        estimator, 
        train[feature], 
        train[target], 
        cv=crossval, 
        scoring='neg_mean_absolute_percentage_error'
        )

    # R2
    r2  =  cross_val_score(
        estimator, 
        train[feature], 
        train[target], 
        cv = crossval, 
        scoring = 'r2'
        )

    # Collect Result
    result.append(
        {
            'Model': name,
            'MAE': abs(mae.mean()),
            'MAPE': abs(mape.mean()),
            'r2':r2.mean(),
            'std-MAE': statistics.stdev(mae),
            'std-MAPE': statistics.stdev(mape),
        }
    )

resultDf = pd.DataFrame(result).set_index('Model')

In [14]:
printedTab = resultDf.sort_values(by='MAE')\
    .style\
    .highlight_max(subset=['r2'], color = 'yellow', axis = 0)\
    .highlight_min(subset=['MAE','MAPE'], color = 'yellow', axis = 0)

# display table
printedTab

,MAE,MAPE,r2,std-MAE,std-MAPE
Model,,,,,
XGBRegressor-logTarget,39643.156334,0.319760,0.374391,1007.554471,0.007080
RandomForestRegressor-logTarget,39784.222167,0.322721,0.368529,928.275160,0.011281
LinearRegression-logTarget,40030.937871,0.334049,0.354400,1731.191266,0.012195
XGBRegressor,40582.754114,0.351155,0.366192,1210.317686,0.014390
LinearRegression,40615.983543,0.373532,0.376050,1080.415640,0.005727
RandomForestRegressor,40624.512110,0.352630,0.364849,1003.156398,0.013468
DecisionTreeRegressor-logTarget,41892.143887,0.349445,0.287329,1136.504437,0.012964
DecisionTreeRegressor,42394.897745,0.369450,0.284902,954.987886,0.012669
KNeighborsRegressor-logTarget,42558.197035,0.346723,0.274074,1586.626377,0.011459


In [15]:
# best model
selected_model = list(resultDf[resultDf['MAE'] == resultDf['MAE'].min()].index.values)[0]
selected_model

'XGBRegressor-logTarget'

### Test Model

In [16]:
model = Pipeline([
    ('preprocessing', ct),
    ('model', models[selected_model])
    ])

model.fit(train[feature], train[target])

pred = model.predict(test[feature])

mae = mean_absolute_error(test[target], pred)
mape = mean_absolute_percentage_error(test[target], pred)
r2 = r2_score(test[target], pred)

score_bt = pd.DataFrame({'MAE': mae, 
                        'MAPE': mape,  
                        'r2': r2}, 
                        index=[selected_model])

In [17]:
printedTab = pd.concat(
    [resultDf.loc[[selected_model]][['MAE','MAPE']], score_bt[['MAE','MAPE']]],
    keys=['Training','Testing'])

printedTab

,,MAE,MAPE
,Model,,
Training,XGBRegressor-logTarget,39643.156334,0.319760
Testing,XGBRegressor-logTarget,39427.293528,0.319711


### Hyperparameter Tuning

In [18]:
# # parameter
# log_xgb.get_params()

In [19]:
# # Kedalaman pohon
# max_depth = list(np.arange(1, 8))
# # Learning rate
# learning_rate = list(np.arange(1, 16)/100)
# # Jumlah pohon
# n_estimators = list(np.arange(100, 1100,100))

# # # Hyperparam space XGboost
# hyperparam_space_xgb = {
#     'model__max_depth': max_depth, 
#     'model__learning_rate': learning_rate,
#     'model__n_estimators': n_estimators,
# }
# for key, val in hyperparam_space_xgb.items():
#     print(key, val)

In [20]:
# estimator_xgb = Pipeline([
#         ('preprocessing', ct),
#         ('model', models[selected_model])
#         ])

# crossval = KFold(n_splits=5, shuffle=True, random_state=7)

# # Hyperparameter tuning
# random_xgb = RandomizedSearchCV(
#     estimator_xgb, 
#     param_distributions = hyperparam_space_xgb,
#     n_iter = 50,
#     cv = crossval, 
#     scoring = ['neg_mean_absolute_error', 'neg_mean_absolute_percentage_error'], 
#     n_jobs = -1,
#     refit = 'neg_mean_absolute_error', # Optimisasi berdasarkan MAE, agar menurunkan tingkat kerugian penjualan
#     random_state = 42  
# )

In [21]:
# random_xgb.fit(train[feature], train[target])

### Save Model

In [22]:
# Fix Model
fix_model = models[selected_model]
fix_model

# Save Model
import pickle

estimator = Pipeline([
        ('preprocessing', ct),
        ('model', fix_model)
        ])

estimator.fit(train[feature], train[target])

pickle.dump(estimator, open('ds-salary-predictor-1.sav', 'wb'))